# Creating the fake database
****

<a id="top"></a>

<b>Table of contents</b><br>

1. [Creating fake population](#population)
2. [Populating the database](#Database)

In this notebook I am going to create a few CSV files with fake data. Then I will use this data to create tables in a new database. The grafic representation (schema) of the database I want to create can be seen below.



![Image of Yaktocat](https://github.com/Manuel-DominguezCBG/SQL2Dashboard/blob/main/Covid-19/Images/screenshot-aca1dabf.png?raw=true)

This database contains 4 interconnected tables. The main table is Patient_data which has basic patient data such as ID, Name, NHS number, gender and so on. In this fake database.

The second table is Covid_19_admission. This table has a proportion of patients from the Patient_data table. The column Patient_admitted_id links with Patient_data.id.


The third table is Covid_19_deaths what is the number of patients that dye a few random days after being admitted. This table has Patient_admited_id column that links with Covid_19_admission.Patient_admitted_id.

The last table is Hospital_features which contained the information of the hospitals. The column Hospital_ID of this table links with Covid_19_death.Hospital_ID and with Covid_19_admission.Hospital_ID.

<a id="population"></a>
## 1. Creating fake people 

In [1]:
# Populating Patient_data dataframe
# 2000 patients with ID number, Name, NHS number, Age, Gender, Ethnicity and postcode.

# Import libraries
import pandas as pd
from pandas import DataFrame
import numpy as np
import random
import datetime
from datetime import timedelta
import names                                     # pip install names
from faker.providers.person.en import Provider   # pip install faker
import sqlite3
from random import randint


size = 2000 # The population of our database

In [2]:
# For creating this database I have taken some info from this dataset
dummy_data=pd.read_csv('https://opendata.arcgis.com/datasets/48b6b85bb7ea43699ee85f4ecd12fd36_0.csv?outSR=%7B%22latestWkid%22%3A27700%2C%22wkid%22%3A27700%7D')
dummy_data.head(3)

# nuts218cd values are the locations of the patients and hospitals of this fake data
# nuts218nm values are going to be the names of the hospitals
# bng_e values are the hospital IDs

,objectid,nuts218cd,nuts218nm,bng_e,bng_n,long,lat,st_areashape,st_lengthshape
0,1,UKC1,Tees Valley and Durham,414643,532242,-1.77440,54.685020,3.026249e+09,487438.014517
1,2,UKC2,Northumberland and Tyne and Wear,398898,602530,-2.01892,55.316841,5.566181e+09,751311.485119
2,3,UKD1,Cumbria,341868,527389,-2.90218,54.638260,6.822195e+09,987066.701024


In [3]:
# Now some funstions to randomly generate our data 
def random_id(size):
    id_patient = random.sample(range(100000000), size)
    return id_patient

In [4]:
def random_NHS_number(size):
    NHS_numbers = random.sample(range(100000000,999999999),size)
    return NHS_numbers

In [5]:
def random_names(name_type, size):
    """
    Generate n-length ndarray of person names.
    name_type: a string, either first_names or last_names
    """
    names = getattr(Provider, name_type)
    return np.random.choice(names, size=size)

In [6]:
def random_genders(size, p=None):
    """Generate n-length ndarray of genders."""
    if not p:
        # Equal proportion of males and females
        p = (0.5, 0.5)
    gender = ("M", "F")
    return np.random.choice(gender, size=size, p=p)

In [7]:
def random_Ethnicity(size, p=None):
    """Generate n-length ndarray of genders."""
    if not p:
        # 5 groups with different probability
        p = (0.49, 0.10, 0.11, 0.01, 0.29)
    Ethnicity = ("White British", "Black British people", "British Indians", "White Gypsy or Irish Traveller", "Other White")
    return np.random.choice(Ethnicity, size=size, p=p)

In [8]:
def random_Postcode(size, p=None):
    if not p:
        # This generate 41 numbers (one per county) which total sum is 1
        p = tuple(map(tuple, np.random.dirichlet(np.ones(41),size=1)))[0]
    Postcode = dummy_data['nuts218cd'].tolist()
    return np.random.choice(Postcode, size=size, p=p)

In [9]:
def random_Hospital_names(size, p=None):
    if not p:
        p = tuple(map(tuple, np.random.dirichlet(np.ones(41),size=1)))[0]
        # Get a list with the name of the counties. Hospitals names = County names
    Hospital_names = dummy_data['nuts218nm'].tolist()
    return np.random.choice(Hospital_names, size=size, p=p)

In [10]:
def random_dates(start, end, size):
    """
    Generate random dates within range between start and end.    
    Adapted from: https://stackoverflow.com/a/50668285
    """
    # Unix timestamp is in nanoseconds by default, so divide it by
    # 24*60*60*10**9 to convert to days.
    divide_by = 24 * 60 * 60 * 10**9
    start_u = start.value // divide_by
    end_u = end.value // divide_by
    return pd.to_datetime(np.random.randint(start_u, end_u, size), unit="D")

## patient_data_df

In [11]:
# Empty df with headers
patients_admitted = pd.DataFrame(columns=['ID', 'NHS_Number','Full_Name','Gender', 'Birthdate', 'Ethnicity', 'Postcode'])

# Populate the dataframe with the functions created above.
patients_admitted['ID'] = random_id(size) 
patients_admitted['NHS_Number'] = random_NHS_number(size)
patients_admitted['first_names'] = random_names('first_names', size)
patients_admitted['last_names'] = random_names('last_names', size)
patients_admitted['Full_Name'] = patients_admitted['first_names']  + ' ' + patients_admitted['last_names']
del patients_admitted['first_names']
del patients_admitted['last_names']
patients_admitted['Gender'] = random_genders(size)
patients_admitted['Birthdate'] = random_dates(start=pd.to_datetime('1936-01-01'), end=pd.to_datetime('2019-01-01'), 
                                              size=size)
patients_admitted['Ethnicity'] = random_Ethnicity(size)
patients_admitted['Postcode'] = random_Postcode(size)
patients_admitted

,ID,NHS_Number,Full_Name,Gender,Birthdate,Ethnicity,Postcode
0,18220033,205662739,Cadence Goodwin,M,2008-07-10,White British,UKD1
1,85808358,193795183,Chandra Keeling,M,1988-05-06,Black British people,UKG3
2,90393718,451650003,Clytie Graham,F,2013-02-12,British Indians,UKF2
3,90184302,888706269,Barrie Turner,M,1981-04-20,White British,UKM7
4,70882403,195404162,Val Grant,F,1969-10-08,Black British people,UKM9
...,...,...,...,...,...,...,...
1995,90706354,495451773,Scottie Barrows,M,2004-08-04,White British,UKI5
1996,19377652,388086154,Jackie Jakubowski,M,1961-08-07,White British,UKH3
1997,42440622,276479135,Alphonsine Schoen,M,1996-06-02,British Indians,UKG1
1998,51612078,885751908,Delton Swift,F,1940-02-12,Other White,UKM7


In [12]:
# To save this as CSV if neccesary

#patients_admitted.to_csv('./patient_data_df.csv')

## covid_19_admission_df

In [13]:
# Populating COVID-19 admission dataframe

# Empty df
covid_19_admission_df = pd.DataFrame(columns=['Patient_admitted_id', 'Date', 'Hospital_ID' ])


# Admission of COVID patients in three hospitals of the same TRUST for one month period

# Populate the Pt_admited_id with the ID of people from patients_admitted
# This table contain the 5% of the patients found in the patient_data_df
patients_admitted_length = int(len(patients_admitted)*0.05)
ID = patients_admitted['ID'].tolist() # ID to list to select the 5% of the values of the ID column
ID2df = (random.choices(ID, k=patients_admitted_length))
covid_19_admission_df['Patient_admitted_id'] = ID2df

#Date from 1 January 2021 to 31 January 2021
covid_19_admission_df['Date'] = random_dates(start=pd.to_datetime('2021-01-01'), 
                                             end=pd.to_datetime('2021-01-31'), size=patients_admitted_length)

# Discharge dates is created by generating random dates and adding to the date of admission
dates = np.random.randint(3,50,len(covid_19_admission_df)).astype('timedelta64[D]')
covid_19_admission_df["Discharge_date"] = covid_19_admission_df['Date'] + dates

# So far, each hospital will get ramdon number of patients
The_hospitals_list = dummy_data['nuts218nm'].tolist()
covid_19_admission_df['Hospital_name'] = np.random.choice(list(The_hospitals_list), len(covid_19_admission_df))

# To add a hospital ID that match always with the name
# Let's create a dict with both values and than populate a new column based on the dict matching 
# Hospital ID with Hospital name

hosp_names2hosp_ID = dict(zip(dummy_data.nuts218nm,dummy_data.bng_e))
# With the dict both columns will match
covid_19_admission_df['Hospital_ID'] = covid_19_admission_df['Hospital_name'].map(hosp_names2hosp_ID)

#Location
hosp_names2location = dict(zip(dummy_data.nuts218nm,dummy_data.nuts218cd))
covid_19_admission_df['Location'] = covid_19_admission_df['Hospital_name'].map(hosp_names2location)


covid_19_admission_df

,Patient_admitted_id,Date,Hospital_ID,Discharge_date,Hospital_name,Location
0,85792580,2021-01-08,376573,2021-02-13,Shropshire and Staffordshire,UKG2
1,35128481,2021-01-13,251287,2021-02-16,Highlands and Islands,UKM6
2,5257399,2021-01-14,462576,2021-02-01,"Berkshire, Buckinghamshire and Oxfordshire",UKJ1
3,52071479,2021-01-12,518883,2021-01-17,Bedfordshire and Hertfordshire,UKH2
4,59307104,2021-01-23,517985,2021-02-19,Lincolnshire,UKF3
...,...,...,...,...,...,...
95,3201344,2021-01-23,344324,2021-01-28,Merseyside,UKD7
96,99860585,2021-01-03,470025,2021-01-16,"Leicestershire, Rutland and Northamptonshire",UKF2
97,85791821,2021-01-29,383347,2021-03-02,"Gloucestershire, Wiltshire and Bath/Bristol area",UKK1
98,27372548,2021-01-17,470025,2021-01-23,"Leicestershire, Rutland and Northamptonshire",UKF2


In [14]:
# To save this as CSV
#covid_19_admission_df.to_csv('./covid_19_admission_df.csv')

## covid_19_death_df

Similar that covid_19_admission_df in which a small proportion of the admitted patients died a few days after admission.


In [15]:
# Crete a dict with ID number and date of admission
id2date_admission = pd.Series(covid_19_admission_df.Date.values,covid_19_admission_df.Patient_admitted_id.values).to_dict()

# Select the 5% of the total number of items  covid_19_admission_df
covid_19_admission_length = int(len(covid_19_admission_df)*0.05)

#Take the 5% of total number of items 
random_entry = random.sample(list(id2date_admission.items()), k=covid_19_admission_length)

# Populate a new df with the patients who are going to die and their date of admission
covid_19_death_df = DataFrame (random_entry,columns=['Patient_admitted_id','Date_admission'])

# Now, we suppose they will die a few random days later, between the 3rd and the 20th day after admission for example.
covid_19_death_df["Death_dates"] = covid_19_death_df["Date_admission"] + timedelta(days=random.randint(3, 20))

# we need the hospital where they were admitted and they died
covid_19_death_df = covid_19_admission_df.merge(covid_19_death_df, on="Patient_admitted_id")

# Some deletion of the columns I dont need
covid_19_death_df = covid_19_death_df.drop(['Date','Date_admission','Discharge_date' ], axis = 1)

# I dont need the date_admission column
covid_19_death_df

,Patient_admitted_id,Hospital_ID,Hospital_name,Location,Death_dates
0,79979929,402507,West Midlands,UKG3,2021-01-27
1,45604406,376573,Shropshire and Staffordshire,UKG2,2021-01-30
2,6396275,414643,Tees Valley and Durham,UKC1,2021-02-01
3,42474557,402507,West Midlands,UKG3,2021-02-04
4,74957191,362838,Dorset and Somerset,UKK2,2021-01-17


## Hospital_features_df
Some characteristics of the hospitals of this fake database

In [16]:
Hospital_features_df = pd.DataFrame( columns=['Hospital_ID','Hospital_name', 'Hospital_location', 
                                                   'Number_of_beds','Number_of_staff', 'Number_of_ITU_Beds' ])

Hospital_features_df['Hospital_ID'] = dummy_data['bng_e']
Hospital_features_df['Hospital_name'] = dummy_data['nuts218nm']
Hospital_features_df['Hospital_location'] = dummy_data['nuts218cd']

Hospital_features_df['Hospital_location'] = Hospital_features_df.assign(Number_of_ITU_Beds=[randint(10, 50) for p in range(41)])

L = [randint(100, 1000) for p in range(41)]
Hospital_features_df['Number_of_beds'] = pd.DataFrame(L)

L = [randint(500, 10000) for p in range(41)]
Hospital_features_df['Number_of_staff'] = pd.DataFrame(L)

L = [randint(50, 100) for p in range(41)]

Hospital_features_df['Number_of_ITU_Beds'] = pd.DataFrame(L)

Hospital_features_df.head(3)

,Hospital_ID,Hospital_name,Hospital_location,Number_of_beds,Number_of_staff,Number_of_ITU_Beds
0,414643,Tees Valley and Durham,414643,112,3789,51
1,398898,Northumberland and Tyne and Wear,398898,813,2866,58
2,341868,Cumbria,341868,865,3353,72


In [17]:
#Hospital_features_df.to_csv('./Hospital_features_df.csv')


<a id="Database"></a>
## Once all CSV have been created, we can use them to populate the database.

In [18]:
# Create a database connection and cursor to execute queries.
conn = sqlite3.connect('./fake_db.db') # This create an empty database in the current directory
c = conn.cursor()

In [19]:
#   1. patient_data_table

### Add and empty table and load patient_data_df into sql table

c.execute('''DROP TABLE IF EXISTS patient_data''')
c.execute(''' CREATE TABLE patient_data (ID NOT NULL,
NHS_Number,
Full_Name,
Gender,
Birthdate,
Ethnicity, 
Postcode,
PRIMARY KEY (ID),
FOREIGN KEY (ID) REFERENCES  covid_19_admission (Patient_admitted_id) ON DELETE CASCADE)''')

# patient_data[ID] ---> covid_19_admission[Patient_admitted_id]
patients_admitted.to_sql('patient_data', conn, if_exists='append', index = False) #LOAD
# c.execute('''SELECT * FROM patient_data''').fetchall() 

In [20]:
#   2. covid_19_admission_table

c.execute('''DROP TABLE IF EXISTS covid_19_admission''')
c.execute(''' CREATE TABLE covid_19_admission (Patient_admitted_id NOT NULL,
Date,
Hospital_ID,
Discharge_date,
Location,
Hospital_name,
FOREIGN KEY (Hospital_ID) REFERENCES  Hospital_features (Hospital_ID) ON DELETE CASCADE,
FOREIGN KEY (Patient_admitted_id) REFERENCES covid_19_death (Patient_admitted_id) ON DELETE CASCADE)''')
covid_19_admission_df.to_sql('covid_19_admission', conn, if_exists='append', index = False)
c.execute('''SELECT * FROM covid_19_admission''').fetchall()

[(85792580,
  '2021-01-08 00:00:00',
  376573,
  '2021-02-13 00:00:00',
  'UKG2',
  'Shropshire and Staffordshire'),
 (35128481,
  '2021-01-13 00:00:00',
  251287,
  '2021-02-16 00:00:00',
  'UKM6',
  'Highlands and Islands'),
 (5257399,
  '2021-01-14 00:00:00',
  462576,
  '2021-02-01 00:00:00',
  'UKJ1',
  'Berkshire, Buckinghamshire and Oxfordshire'),
 (52071479,
  '2021-01-12 00:00:00',
  518883,
  '2021-01-17 00:00:00',
  'UKH2',
  'Bedfordshire and Hertfordshire'),
 (59307104,
  '2021-01-23 00:00:00',
  517985,
  '2021-02-19 00:00:00',
  'UKF3',
  'Lincolnshire'),
 (63291075,
  '2021-01-10 00:00:00',
  516574,
  '2021-02-23 00:00:00',
  'UKI7',
  'Outer London - West and North West'),
 (96476592,
  '2021-01-04 00:00:00',
  300521,
  '2021-01-15 00:00:00',
  'UKM7',
  'Eastern Scotland'),
 (75682054,
  '2021-01-07 00:00:00',
  414643,
  '2021-01-22 00:00:00',
  'UKC1',
  'Tees Valley and Durham'),
 (82129989,
  '2021-01-05 00:00:00',
  383347,
  '2021-01-18 00:00:00',
  'UKK1',
  

In [21]:
#   3. covid_19_death_table#   4. Hospital_features_df

c.execute('''DROP TABLE IF EXISTS Hospital_features''')
c.execute(''' CREATE TABLE Hospital_features ( Hospital_ID NOT NULL PRIMARY KEY,
Hospital_name,
Hospital_location,
Number_of_beds number (3),
Number_of_staff number (3),
Number_of_ITU_Beds number (3))''')
Hospital_features_df.to_sql('Hospital_features', conn, if_exists='append', index = False)
c.execute('''SELECT * FROM Hospital_features''').fetchall()

c.execute('''DROP TABLE IF EXISTS covid_19_death''')
c.execute(''' CREATE TABLE covid_19_death (Patient_admitted_id NOT NULL,
Hospital_ID,
Death_dates,
Hospital_name,
Location,
PRIMARY KEY (Patient_admitted_id),
FOREIGN KEY (Hospital_ID) REFERENCES  Hospital_features (Hospital_ID) ON DELETE CASCADE)''')
covid_19_death_df.to_sql('covid_19_death', conn, if_exists='append', index = False)
c.execute('''SELECT * FROM covid_19_death''').fetchall()

[(79979929, 402507, '2021-01-27 00:00:00', 'West Midlands', 'UKG3'),
 (45604406,
  376573,
  '2021-01-30 00:00:00',
  'Shropshire and Staffordshire',
  'UKG2'),
 (6396275, 414643, '2021-02-01 00:00:00', 'Tees Valley and Durham', 'UKC1'),
 (42474557, 402507, '2021-02-04 00:00:00', 'West Midlands', 'UKG3'),
 (74957191, 362838, '2021-01-17 00:00:00', 'Dorset and Somerset', 'UKK2')]

In [22]:
#   4. Hospital_features_df

c.execute('''DROP TABLE IF EXISTS Hospital_features''')
c.execute(''' CREATE TABLE Hospital_features ( Hospital_ID NOT NULL PRIMARY KEY,
Hospital_name,
Hospital_location,
Number_of_beds number (3),
Number_of_staff number (3),
Number_of_ITU_Beds number (3))''')
Hospital_features_df.to_sql('Hospital_features', conn, if_exists='append', index = False)
c.execute('''SELECT * FROM Hospital_features''').fetchall()

[(414643, 'Tees Valley and Durham', 414643, 112, 3789, 51),
 (398898, 'Northumberland and Tyne and Wear', 398898, 813, 2866, 58),
 (341868, 'Cumbria', 341868, 865, 3353, 72),
 (377940, 'Greater Manchester', 377940, 920, 2071, 69),
 (363669, 'Lancashire', 363669, 278, 680, 56),
 (368759, 'Cheshire', 368759, 456, 5369, 69),
 (344324, 'Merseyside', 344324, 624, 9102, 54),
 (488051, 'East Yorkshire and Northern Lincolnshire', 488051, 467, 4046, 58),
 (429509, 'North Yorkshire', 429509, 993, 1783, 73),
 (441052, 'South Yorkshire', 441052, 565, 1146, 94),
 (418686, 'West Yorkshire', 418686, 452, 9078, 80),
 (449616, 'Derbyshire and Nottinghamshire', 449616, 419, 9142, 68),
 (470025,
  'Leicestershire, Rutland and Northamptonshire',
  470025,
  550,
  5759,
  89),
 (517985, 'Lincolnshire', 517985, 514, 4439, 59),
 (390692,
  'Herefordshire, Worcestershire and Warwickshire',
  390692,
  530,
  7395,
  69),
 (376573, 'Shropshire and Staffordshire', 376573, 712, 1987, 75),
 (402507, 'West Midlan

## Database created. 

In [23]:
# Let's ensure everything is ok.
# The encoding pragma controls how strings are encoded and stored in a database file.
c.execute("PRAGMA table_info(patient_data);").fetchall()
# id 	name 	type 	notnull 	dflt_value 	pk

[(0, 'ID', '', 1, None, 1),
 (1, 'NHS_Number', '', 0, None, 0),
 (2, 'Full_Name', '', 0, None, 0),
 (3, 'Gender', '', 0, None, 0),
 (4, 'Birthdate', '', 0, None, 0),
 (5, 'Ethnicity', '', 0, None, 0),
 (6, 'Postcode', '', 0, None, 0)]

In [24]:
c.execute("PRAGMA table_info(covid_19_admission);").fetchall()

[(0, 'Patient_admitted_id', '', 1, None, 0),
 (1, 'Date', '', 0, None, 0),
 (2, 'Hospital_ID', '', 0, None, 0),
 (3, 'Discharge_date', '', 0, None, 0),
 (4, 'Location', '', 0, None, 0),
 (5, 'Hospital_name', '', 0, None, 0)]

In [25]:
c.execute("PRAGMA table_info(covid_19_death);").fetchall()

[(0, 'Patient_admitted_id', '', 1, None, 1),
 (1, 'Hospital_ID', '', 0, None, 0),
 (2, 'Death_dates', '', 0, None, 0),
 (3, 'Hospital_name', '', 0, None, 0),
 (4, 'Location', '', 0, None, 0)]

In [26]:
c.execute("PRAGMA table_info(Hospital_features);").fetchall()

[(0, 'Hospital_ID', '', 1, None, 1),
 (1, 'Hospital_name', '', 0, None, 0),
 (2, 'Hospital_location', '', 0, None, 0),
 (3, 'Number_of_beds', 'number (3)', 0, None, 0),
 (4, 'Number_of_staff', 'number (3)', 0, None, 0),
 (5, 'Number_of_ITU_Beds', 'number (3)', 0, None, 0)]

### Notebook details
<br>
<i>Notebook created by <strong>Manuel Dominguez</strong> 

Creation date: May 2021<br>


Code to create the sheme in Database schema 

//// -- Tables and References

// Creating tables
Table Patient_data {
  id int [pk, increment] // auto-increment
  Full_name varchar
  NHS_number int
  Birthdate int
  Gender varchar
  Ethnicity varchar
  Postcode varchar
}

Table Covid_19_admission {
  Patients_admitted_id int [ref: > Patient_data.id]  // inline relationship (many-to-one)
  Date_adm int
  Hospital_ID varchar [ref: > Hospital_features.Hospital_ID]
  Indexes {
    (Patients_admitted_id) [pk]
  }
}



Table Hospital_features {
 Hospital_ID varchar
 Hospital_name varchar 
 Hospital_location varchar
 Number_of_beds varchar
 Number_of_ITU_beds varchar
 

}



Table covid_19_death {
 Patients_admitted_id  int [ref: > Covid_19_admission.Patients_admitted_id]
 Hospital_ID varchar [ref: > Hospital_features.Hospital_ID]
 Death_dates varchar 
 Indexes {
    (Patients_admitted_id) [pk]
  }
 

}